# Instructions of this exercise

This jupyter notebook is an exercise for you to understand the basic word2vec skipgram model, as well as other foundations in NLP tasks, such as corpus preprocessing, model training and inference (with pytorch).

The different sections have comments that will guide you through the different stages of the processing.

**You will find parts in this notebook that are missing. They are marked with a <font color="red">"TODO"</font>, with an explanation about what you are supposed to write/code in that gap. Sometimes you may be requested to implement an algorithm or a piece of code that behaves in a specific way. Other times, you may be requested to explain what a specific piece of code does or why it is needed.** In some cases, the requests may contain a "trap", e.g. the thing you are requested to implement can't be done for whatever reason, or it may need an extra preprocessing step for it to be possible.

If you have any doubt, do not hesitate to contact the instructor at noe.casas@upc.edu

---



# Requirements

In order to follow this notebook you need working knowledge on:

* Python programming.
* Pytorch basics.
* Word embedding basics.
* Unix command line basics (cat, head, tail, awk).

If you lack any of them, please acquire the needed knowledge on the internet, where there is plenty of material about the three topics.

---

# Introduction

In this jupyter notebook, we will illustrate how the word2vec **skipgram** model
works. We will be using the 2 word2vec original articles as guidelines to
implement a simplified version of it:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781)
* [Distributed Representations of Words and Phrases and their Compositionality](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality)

Word2vec is an approach to create word embeddings. It proposes two variants:
* Continuous Bag of Words (CBoW)
* Skipgram

In both cases, word2vec proposes to use a shallow neural network trained on
textual data. After the training, we simply take the resulting word embeddings
and use them in the downstream task of our choice.

## Word2vec Algorithms

The **Continuous Bag of Words (CBoW)** word2vec variant proposes to train a
neural net to predict a word taking as input its surrounding words up to a certain distance. This way, assuming a maximum distance of 2, for each word $w_t$ in our training text (where $t$ is the position of the word), we will create 1 training sample where the expected output is $w_t$ and the input is $w_{t-2}, w_{t-1}, w_{t+1}, w_{t+2}$.

The **Skipgram** word2vec variant proposes to train a neural net to predict the surrounding words (up to a certain distance) taking as input the word itself. This way, assuming a maximum distance of 2, for each word $w_t$ in our training text, we will create 4 training samples where the input is $w_t$ and the expected outputs are each of $w_{t-2}, w_{t-1}, w_{t+1}, w_{t+2}$.

The skipgram model can be applied extra elements that improve the quality of the embeddings. **These improvements ARE NOT USED IN THIS NOTEBOOK**. Nevertheless -just to satisfy your curiosity- they are:
* Resampling based on distance to the focus word. See [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/abs/1301.3781) section 3.2:
> Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples.
* Hierarchical softmax. See [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality) section 2.1.
* Negative sampling. See [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality) section 2.2.
* Subsampling of Frequent Words. See [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality) section 2.3.


## Hardware Resources

Let's have a look at the GPU we will be using:

In [5]:
!nvidia-smi

Tue Mar 24 22:36:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

---
## Installation of extra libraries

**NOTE**: In jupyter, apart from being able to execute Python code, you can also run commands in the command line by preceding the invocation with a "`!`". For instance, if you want to execute "`ls`", you may simply write `!ls`

While Google Colab has a lot of libraries installed by default, we need some extra ones. In order to install them, we will use `pip`, executing it from the command line with `!`.

The extra libraries we will use are:
* **sacremoses**: the is a classical translation tool called "Moses" (https://github.com/moses-smt/mosesdecoder). Apart from translation tools, it offers a lot of different preprocessing scripts (written in the Perl programming language). These scripts are ubiquitous in the NLP world. Instead of using Moses scripts, we will use sacremoses, which is a reimplementation of the most useful Moses scripts in Python.

In [6]:
!pip install -U https://github.com/alvations/sacremoses/archive/master.zip

  Using cached https://github.com/alvations/sacremoses/archive/master.zip
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=883294 sha256=480b258c8eb876758c04bce8acab77481192d237bf2a5ee59fa7d7d5fb96a090
  Stored in directory: /tmp/pip-ephem-wheel-cache-l68xkp95/wheels/a9/8b/db/e688b9d5eecd3518dd27712980c76599bcd8312ee220f4878e
Successfully built sacremoses
  Found existing installation: sacremoses 0.0.38
    Uninstalling sacremoses-0.0.38:
      Successfully uninstalled sacremoses-0.0.38


---

## General Imports

Now we will import some components that will be used throughout the notebook

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

---

## Reproducibility

We fix the random seed for reproducibility purposes. See this for details:
https://pytorch.org/docs/stable/notes/randomness.html

In [0]:
seed = 321
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

---

## Data Download

For this notebook we will use a dataset widely used in machine translation, called "News Commentary". We will be using some English subsets of News Commentary as training, validation and test data.

First, we will download the dataset from the internet and extract some parts into files `train.en`, `valid.en` and `test.en`:

In [0]:
!wget --quiet 'http://data.statmt.org/wmt17/translation-task/training-parallel-nc-v12.tgz'
!tar xzf training-parallel-nc-v12.tgz
!mv training/news-commentary-v12.fr-en.en ./train.en
!cat training/news-commentary-v12.zh-en.en | shuf --random-source=<(openssl enc -aes-256-ctr -pass pass:42 -nosalt </dev/zero 2>/dev/null) > shuffled.en
!head -5000 < shuffled.en > valid.en
!tail -5000 < shuffled.en > test.en

News commentary is text from news, where each line of the file is a separate sentence. Let's take a look at the number of sentences in each file:

In [10]:
!wc -l train.en valid.en test.en

  258432 train.en
    5000 valid.en
    5000 test.en
  268432 total


---


## Data preprocessing

In general, it is of utmost importance to understand the data you are working with, so let's have a look at the text:

In [11]:
!head -5 train.en

$10,000 Gold?
SAN FRANCISCO – It has never been easy to have a rational conversation about the value of gold.
Lately, with gold prices up more than 300% over the last decade, it is harder than ever.
Just last December, fellow economists Martin Feldstein and Nouriel Roubini each penned op-eds bravely questioning bullish market sentiment, sensibly pointing out gold’s risks.
Wouldn’t you know it?


As a first preprocessing step, we will:
* **normalize punctuation**: convert punctuation marks so that only the most standard ones are used. For instance, unicode curly quotes (“”) will be converted to straigh ones (").
* **tokenize**: words and punctuation will be separated with spaces, and contractions may be expanded. For instance, sentence "No, he wasn't there." will be converted to "No , he was n't there ."
* **lowercase**: turn all alphabetical characters into lowercase, so that the embeddings we obtain are independent from the original casing.

In [12]:
!cat train.en | sacremoses normalize -l en | sacremoses tokenize -l en | awk '{ print tolower($0) }' > train.low.tok.en
!cat valid.en | sacremoses normalize -l en | sacremoses tokenize -l en | awk '{ print tolower($0) }' > valid.low.tok.en
!cat test.en  | sacremoses normalize -l en | sacremoses tokenize -l en | awk '{ print tolower($0) }' > test.low.tok.en

100% 258961/258961 [00:30<00:00, 8606.15it/s]
100% 258961/258961 [00:41<00:00, 6270.10it/s]
100% 5000/5000 [00:00<00:00, 12124.99it/s]
100% 5000/5000 [00:00<00:00, 5947.36it/s]
100% 5004/5004 [00:00<00:00, 11133.39it/s]
100% 5004/5004 [00:00<00:00, 5257.56it/s]


Let's look how our data looks like after the described preprocessing steps:

In [13]:
!head -10 train.low.tok.en

$ 10,000 gold ?
san francisco - it has never been easy to have a rational conversation about the value of gold .
lately , with gold prices up more than 300 % over the last decade , it is harder than ever .
just last december , fellow economists martin feldstein and nouriel roubini each penned op-eds bravely questioning bullish market sentiment , sensibly pointing out gold &apos;s risks .
wouldn &apos;t you know it ?
since their articles appeared , the price of gold has moved up still further . gold prices even hit a record-high $ 1,300 recently .
last december , many gold bugs were arguing that the price was inevitably headed for $ 2,000 .
now , emboldened by continuing appreciation , some are suggesting that gold could be headed even higher than that .
one successful gold investor recently explained to me that stock prices languished for a more than a decade before the dow jones index crossed the 1,000 mark in the early 1980 &apos;s .
since then , the index has climbed above 10,000 .


Now we will **mask the numbers** in the text with a "NUM" placeholder. We do this because we don't want to create embeddings of the numbers present in the text.

In [0]:
!cat train.low.tok.en | sed 's/[0-9]*[.,0-9]*[0-9]/NUM/g' > train.low.tok.nonum.en
!cat valid.low.tok.en | sed 's/[0-9]*[.,0-9]*[0-9]/NUM/g' > valid.low.tok.nonum.en
!cat test.low.tok.en  | sed 's/[0-9]*[.,0-9]*[0-9]/NUM/g' > test.low.tok.nonum.en

In [15]:
!head -10 train.low.tok.nonum.en

$ NUM gold ?
san francisco - it has never been easy to have a rational conversation about the value of gold .
lately , with gold prices up more than NUM % over the last decade , it is harder than ever .
just last december , fellow economists martin feldstein and nouriel roubini each penned op-eds bravely questioning bullish market sentiment , sensibly pointing out gold &apos;s risks .
wouldn &apos;t you know it ?
since their articles appeared , the price of gold has moved up still further . gold prices even hit a record-high $ NUM recently .
last december , many gold bugs were arguing that the price was inevitably headed for $ NUM .
now , emboldened by continuing appreciation , some are suggesting that gold could be headed even higher than that .
one successful gold investor recently explained to me that stock prices languished for a more than a decade before the dow jones index crossed the NUM mark in the early NUM &apos;s .
since then , the index has climbed above NUM .


Finally, we will **remove the punctuation** from the text, as we just want to compute word embeddings, nor including commas, stops or quotes.

In [0]:
!cat train.low.tok.nonum.en | tr -d '[:punct:]' | sed 's,[ ]\+, ,g' > train.low.tok.nonum.nopunct.en
!cat valid.low.tok.nonum.en | tr -d '[:punct:]' | sed 's,[ ]\+, ,g' > valid.low.tok.nonum.nopunct.en
!cat test.low.tok.nonum.en | tr -d '[:punct:]' | sed 's,[ ]\+, ,g' > test.low.tok.nonum.nopunct.en

In [17]:
!head -10 train.low.tok.nonum.nopunct.en

 NUM gold 
san francisco it has never been easy to have a rational conversation about the value of gold 
lately with gold prices up more than NUM over the last decade it is harder than ever 
just last december fellow economists martin feldstein and nouriel roubini each penned opeds bravely questioning bullish market sentiment sensibly pointing out gold aposs risks 
wouldn apost you know it 
since their articles appeared the price of gold has moved up still further gold prices even hit a recordhigh NUM recently 
last december many gold bugs were arguing that the price was inevitably headed for NUM 
now emboldened by continuing appreciation some are suggesting that gold could be headed even higher than that 
one successful gold investor recently explained to me that stock prices languished for a more than a decade before the dow jones index crossed the NUM mark in the early NUM aposs 
since then the index has climbed above NUM 


Now we have our 3 final files: `train.low.tok.nonum.nopunct.en`, `valid.low.tok.nonum.nopunct.en` and `test.low.tok.nonum.nopunct.en`.

Note that, at each preprocessing step, we have been creating new files with extra suffixes indicating the type of preprocessing.

---

## Text Loading

We will load the text from our 3 files into python variables. The text in a file will be represented as a list of sentences, where each sentence is a list of words (strings).

In [0]:
def load_text_tokens(filename):
    """
    Loads the text in the filename as a list of list of words.
    :param filename Name of the file to load in memory.
    :return List of list of strings with the contents of the file.
    """
    text = []
    with open(filename) as f:
        for line in f:
            line = line.strip() # remove leading and training blanks

            # split words at blanks (the text is already tokeniked)
            line_tokens = line.split(' ')

            if len(line_tokens) < 5:
              continue  # ignore too short lines

            text.append(line_tokens)
    return text

In [0]:
train_tokens = load_text_tokens('train.low.tok.nonum.nopunct.en')
valid_tokens = load_text_tokens('valid.low.tok.nonum.nopunct.en')
test_tokens = load_text_tokens('test.low.tok.nonum.nopunct.en')

## Vocabulary Extraction

Given the huge amount of possible word in a language, when creating word embeddings, we need to constrain the set of supported words. This way, we will be having a list of words for which we will compute embeddings. This list is known as the **vocabulary**. The vocabulary is created by selecting the most frequent words in the training data.

As the vocabulary is finite, there are some words it won't be able to represent. We will use a **special token** to represent these "out of vocabulary" words, normally referred to as the **<UNK>** token.

In [0]:
def extract_vocabulary(text_tokens, vocab_size):
    from collections import Counter
    unk_token = '<UNK>'
    special_tokens = [unk_token]
    counter = Counter(word for sentence in text_tokens for word in sentence)
    most_common_counts = counter.most_common(vocab_size - len(special_tokens))
    most_frequent_words = [word for (word, count) in most_common_counts]
    idx2token = special_tokens + most_frequent_words
    token2idx = {token: token_id for token_id, token in enumerate(idx2token)}
    return unk_token, idx2token, token2idx

In [0]:
unk_token, idx2token, token2idx = extract_vocabulary(train_tokens, vocab_size=50000)
unk_token_id = token2idx[unk_token]

# Text Encoding

To feed words to our neural network, we need to express them as numbers. This way, a word will be represented as the index it occupies in the vocabulary table. With this in mind, let's encoder the training, validation and test datasets:


In [0]:
def encode_token_ids(text_tokens, vocabulary, unk_token_id):
    return [[vocabulary.get(token, unk_token_id) for token in sentence]
            for sentence in text_tokens]

In [0]:
train_token_ids = encode_token_ids(train_tokens, token2idx, unk_token_id)
valid_token_ids = encode_token_ids(test_tokens, token2idx, unk_token_id)
test_token_ids = encode_token_ids(test_tokens, token2idx, unk_token_id)

---

## Preparation of the data for the skipgram model


Now we will prepare the dataset according to the skipgram approach:

In [0]:
def create_skipgram_dataset(token_ids, unk_token_id, window):
    """

    :param token_ids: token IDs to be adapted to the skipgram model. It is 
                      a list of lists of token IDs (integers).
    :param unk_token_id token ID of the <UNK> token.
    :param window distance in words to the left/right that the context will include.
    :return A list of tuples (input_token_id, output_token_id). No tuples with
            the unk_token_id are returned.
    """
    data = []
    for sentence_token_ids in token_ids:
        # TODO: implement the code to get the training data (inputs and
        #       expected outputs) according to the skipgram model. You can
        #       review the basics of skipgrams in the introduction above,
        #       and you can also check the assertions below to better understand
        #       what this function is supposed to receive and generate as
        #       output.
        for i in range(window, len(sentence_token_ids) - window):
            if sentence_token_ids[i] == unk_token_id:
                continue

            for j in range(i - window, i + window + 1):
                if (sentence_token_ids[j] == unk_token_id) or (j == i):
                    continue 
                data.append((sentence_token_ids[i], sentence_token_ids[j]))
    return data

These "unit tests" may help to 1) understand what it is expected from the function and 2) validate the function.

In [25]:
create_skipgram_dataset([[1, 2, 3, 4, 5]], 0, 2)

[(3, 1), (3, 2), (3, 4), (3, 5)]

In [0]:
assert create_skipgram_dataset([[1, 2, 3, 4, 5]], 0, 2) == [(3, 1), (3, 2), (3, 4), (3, 5)]
assert create_skipgram_dataset([[1, 2, 3, 4, 5, 6]], 0, 2) == [(3, 1), (3, 2), (3, 4), (3, 5), (4, 2), (4, 3), (4, 5), (4, 6)]
assert create_skipgram_dataset([[1, 2, 3, 0, 5, 6]], 0, 2) == [(3, 1), (3, 2), (3, 5)]
assert create_skipgram_dataset([[1, 0, 3, 4, 5]], 0, 2) == [(3, 1), (3, 4), (3, 5)]
assert create_skipgram_dataset([[1, 0, 3, 4, 5, 0, 7]], 0, 3) == [(4, 1), (4, 3), (4, 5), (4, 7)]

Now, let's prepare the actual datasets:

In [0]:
window_size = 2
skipgram_train = create_skipgram_dataset(train_token_ids, unk_token_id, window=window_size)
skipgram_valid = create_skipgram_dataset(valid_token_ids, unk_token_id, window=window_size)
skipgram_test = create_skipgram_dataset(test_token_ids, unk_token_id, window=window_size)

## Neural Network

Now we start with the Pytorch stuff. First, the model. Actually, it's the model together with the loss function:

In [0]:
class SkipgramLoss(nn.Module):
    """
    Network that implements the basic Skipgram word2vec, without the negative
    sampling loss or hierarchical softmax.
    """

    def __init__(self, vocab_size, embedding_size):
        """
        Constructor.
        :param vocab_size Size of the vocabulary.
        :param embedding_size Size of the embedding vectors.
        """
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, input, output):
        """
        :param input LongTensor with shape [batch_size, input_size]
        :param output LongTensor with shape [batch_size]
        """
        v_wi = self.embedding(input)    # [bsz, embsz]
        v_all = self.embedding.weight   # [vocab_size, embsz]

        # TODO: implement the computation of the logits of the softmax according
        #       to equation (2) in the article "Distributed Representations of
        #       Words and Phrases and their Compositionality".
        #       (http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality)
        logits = v_wi @ v_all.T

        return self.cross_entropy(logits, output)
    
    def get_embedding_table(self):
        return self.embedding.weight

Now, let's define a function that trains the model for a whole epoch:

In [0]:
def train_epoch(train_data_loader, loss_function, optimizer, log_period, epoch):
  """
  Trains the given model for 1 epoch.
  :param train_data_loader Data loader of the training data. Batches must be
                           a tuple of input and output data.
  :param loss_function Loss function to train.
  :param optimizer Optimizer object to use.
  :param log_period Number of steps after which to print a trace.
  :param epoch Epoch to print in the traces.
  :return List of losses for every training step.
  """
  loss_function.train()

  num_total_data = len(train_data_loader.dataset)
  num_processed_data = 0

  losses = []
  for batch_ndx, batch in enumerate(train_data_loader):
      input, output = batch
      optimizer.zero_grad()
      loss = loss_function(input.cuda(), output.cuda())
      loss.backward()
      optimizer.step()
      loss_value = loss.item()
      losses.append(loss_value)
      num_processed_data += batch[0].size(0)

      if log_period and batch_ndx % log_period == 0:
          percentage = 100 * num_processed_data // num_total_data
          avg_loss = loss_value if batch_ndx == 0 else np.mean(losses[-log_period + 1:])
          msg = "epoch: {}, progress: {}%, avg.loss: {:.3f}"
          print(msg.format(epoch, percentage, avg_loss))
  
  return losses

Now, a function to evaluate our model in the validation data:

In [0]:

def eval_loss(data_loader, loss_function):
    """
    Computes the loss function over the given data in inference mode.
    :param data_loader Data loader over which to evaluate loss_function.
    :param loss_function Loss function to evaluate.
    :return Value (float) of the loss.
    """
    loss_function.eval()
    
    num_total_data = len(data_loader.dataset)

    with torch.no_grad():
        loss = 0.0
        for batch in data_loader:
            input, output = batch
            bsz = input.size(0)
            batch_loss = loss_function(input.cuda(), output.cuda())
            
            # we average over all batches
            loss += bsz * batch_loss.item() / num_total_data
    
    return loss

Finally, a function to create and train the embeddings for N epochs, evaluating the model every epoch:

In [0]:
def train_embedding(train_data,
                    valid_data,
                    vocab_size,
                    emb_size,
                    num_epochs,
                    batch_size,
                    log_period=None):
  """
  Trains embeddings with the given parameters.
  :param train_data Training data (list of tuples of input and output token IDs).
  :param valid_data Validation data (list of tuples of input and output token IDs).
  :param vocab_size Size of the vocabulary.
  :param emd_size Dimensionality of the embedding.
  :param num_epochs Number of epochs to train.
  :param batch_size Batch size to use.
  :param log_period Number of steps after which to print a trace.
  :return A tupe with the embedding tensor, the list of the training
          losses and the list of the validation losses.
  """

  # Load the whole training a validation data as pytorch tensors
  train_input = torch.from_numpy(np.array([input for (input, __) in train_data]))
  train_output = torch.from_numpy(np.array([output for (_, output) in train_data]))
  valid_input = torch.from_numpy(np.array([input for (input, _) in valid_data]))
  valid_output = torch.from_numpy(np.array([output for (_, output) in valid_data]))

  # Define data loaders for the training and validation data
  train_loader = DataLoader(TensorDataset(train_input, train_output),
                            batch_size=batch_size,
                            pin_memory=True)
  valid_loader = DataLoader(TensorDataset(valid_input, valid_output),
                            batch_size=batch_size,
                            pin_memory=True)

  # Create the model (+ loss function) and move it to the GPU
  loss_function = SkipgramLoss(vocab_size, emb_size).cuda()

  # Define the optimizer and LR scheduler
  optimizer = optim.Adam(lr=0.005, params=loss_function.parameters())
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

  # Define lists to collect the training and validation losses in case
  # we want to plot them later.
  train_losses = []
  valid_losses = []

  # Iterate the requested number of epochs
  for epoch in range(num_epochs):
      # Train for one epoch
      train_losses.extend(train_epoch(train_loader,
                                      loss_function,
                                      optimizer,
                                      log_period,
                                      epoch))

      # Evaluate the model on the validation data
      valid_loss = eval_loss(valid_loader, loss_function)
      valid_losses.append(valid_loss)
      
      scheduler.step()

      if log_period:
          # Print traces every once in a while
          print("epoch: {}, validation loss: {:.3f}".format(epoch, valid_loss))
  
  # This is the embedding table that we have been training
  embedding = loss_function.get_embedding_table()

  return embedding, train_losses, valid_losses

And now we actually train the embeddings:

In [32]:
embedding, train_losses, valid_losses = train_embedding(skipgram_train,
                                                        skipgram_valid,
                                                        vocab_size=len(idx2token),
                                                        emb_size=64,
                                                        num_epochs=3,
                                                        batch_size=4000,
                                                        log_period=400)

epoch: 0, progress: 0%, avg.loss: 64.786
epoch: 0, progress: 8%, avg.loss: 35.255
epoch: 0, progress: 16%, avg.loss: 16.143
epoch: 0, progress: 25%, avg.loss: 12.199
epoch: 0, progress: 33%, avg.loss: 9.967
epoch: 0, progress: 42%, avg.loss: 9.164
epoch: 0, progress: 50%, avg.loss: 8.651
epoch: 0, progress: 59%, avg.loss: 8.339
epoch: 0, progress: 67%, avg.loss: 8.044
epoch: 0, progress: 76%, avg.loss: 7.840
epoch: 0, progress: 84%, avg.loss: 7.670
epoch: 0, progress: 93%, avg.loss: 7.659
epoch: 0, validation loss: 7.481
epoch: 1, progress: 0%, avg.loss: 8.185
epoch: 1, progress: 8%, avg.loss: 7.916
epoch: 1, progress: 16%, avg.loss: 7.400
epoch: 1, progress: 25%, avg.loss: 7.466
epoch: 1, progress: 33%, avg.loss: 7.296
epoch: 1, progress: 42%, avg.loss: 7.350
epoch: 1, progress: 50%, avg.loss: 7.356
epoch: 1, progress: 59%, avg.loss: 7.357
epoch: 1, progress: 67%, avg.loss: 7.324
epoch: 1, progress: 76%, avg.loss: 7.286
epoch: 1, progress: 84%, avg.loss: 7.247
epoch: 1, progress: 93%,

---

## Explore the Embedded Vectors

First, let's define a function to find the list of K closes vectors:

In [0]:
def find_closest_ids(embedding, vector, k):
    """
    Finds the closes K vectors to the given one among the given embedding matrix.
    :param embedding Embedding matrix.
    :param vector Vector to which we should find the closes ones in the matrix.
    :param k Number of closest elements to find.
    """
    similarity = F.cosine_similarity(embedding, vector.unsqueeze(0))
    index_sorted = torch.argsort(similarity).cpu().numpy()
    top_k = index_sorted[-k:]
    return list(reversed(top_k))

Let's find the words that are close to some target words:

In [40]:
words = ["minister", "France", "measures"]

for word in words:
    #TODO: print the closes 5 words to each of the words in the list idx2token, token2idx
    word_vector = embedding[token2idx[word.lower().strip()]]
    similar_token_ids = find_closest_ids(embedding, word_vector, 6)
    similar_words = [idx2token[id] for id in similar_token_ids[1:]]
    print("Similar words to {}: {}".format(word, similar_words))

Similar words to minister: ['prime', 'former', 'british', 'david', 'deputy']
Similar words to France: ['germany', 'italy', 'britain', 'russia', 'japan']
Similar words to measures: ['policies', 'implement', 'such', 'reforms', 'austerity']


The "geometry" of word embedded spaces has been widely studied, observing the  "parallelogram" formed by the vectors of two pairs of words with analogous relationships. You can read more about this in this recent ICML article: http://proceedings.mlr.press/v97/allen19a.html


In [0]:
# TODO: making use of the function find_closest_ids, try to find two pairs of
#       words that verify the parallelogram in embedding space and two pairs of
#       words that do not verify it.
def a_is_to_b_as_c_is_to(a, b, c):
    a_vector = embedding[token2idx[a.lower()]]
    b_vector = embedding[token2idx[b.lower()]]
    c_vector = embedding[token2idx[c.lower()]]
    new_vector = b_vector - a_vector + c_vector
    closest_id = find_closest_ids(embedding, new_vector, 1)
    return idx2token[closest_id[0]]


In [74]:
a_is_to_b_as_c_is_to("man", "king", "woman")

'king'

In [75]:
a_is_to_b_as_c_is_to("man", "waiter", "woman")

'waiter'

In [76]:
a_is_to_b_as_c_is_to("woman", "mom", "man")

'dies'

In [77]:
a_is_to_b_as_c_is_to("spain", "madrid", "france")

'madrid'

Nevertheless, be aware that many of the parallelogram examples in the literature (specially those that were considered to show gender bias) where due to an implementation detail in gensim, which explicitly avoided the returned vector to be the same as the original one. Read about this at https://twitter.com/rikvannoord/status/1132933236756238341


**TODO**: explain the misunderstanding that amplified our belief in the presence of biases in word embeddings.

---

This is not to say that our data, models or resulting word embeddings are bias free. On the contrary, there is **gender bias** in the three of them. Check the literature on gender bias in word embeddings to know more. You can start with these two seminal works:
* [Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings](https://papers.nips.cc/paper/6228-man-is-to-computer-programmer-as-woman-is-to-homemaker-debiasing-word-embeddings)
* [Lipstick on a Pig: Debiasing Methods Cover up Systematic Gender Biases in Word Embeddings But do not Remove Them](https://www.aclweb.org/anthology/N19-1061/)

---